<a id='top'></a><a name='top'></a>
# Chapter 10: Sequence-to-sequence models and attention

## LSTM models

Code is based on this notebook: [A ten-minute introduction to sequence-to-sequence learning in Kera](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html) by François Chollet

* [Introduction](#introduction)
* [10.0 Imports and Setup](#10.0)
* [10.1 Code](#10.1)



<!-- * [Introduction](#introduction)
* [10.0 Imports and Setup](#9.0)
* [10.1 Encoder-decoder architecture](#10.1)
    - [10.1.1 Decoding thought](#10.1.1)
    - [10.1.2 Look familiar?](#10.1.2)
    - [10.1.3 Sequence-to-sequence conversation](#10.1.3)
    - [10.1.4 LSTM review](#10.1.4)
* [10.2 Assembling a sequence-to-sequence pipeline](#10.2)
    - [10.2.1 Preparing your dataset for the sequence-to-sequence training](#10.2.1)
    - [10.2.2 Sequence-to-sequence model in Keras](#10.2.2)
    - [10.2.3 Sequence encoder](#10.2.3)
    - [10.2.4 Thought decoder](#10.2.4)
    - [10.2.5 Assembling the sequence-to-sequence network](#10.2.5)
* [10.3 Training the sequence-to-sequence network](#10.3)
    - [10.3.1 Generate output sequences](#10.3.1)
* [10.4 Building a chatbot using sequence-to-sequence networks](#10.4)
    - [10.4.1 Preparing the corpus for your training](#10.4.1)
    - [10.4.2 Building your character dictionary](#10.4.2)
    - [10.4.3 Generate one-hot encoded training sets](#10.4.3)
    - [10.4.4 Train your sequence-to-sequence chatbot](#10.4.4)
    - [10.4.5 Assemble the model for sequence-to-sequence generation](#10.4.5)
    - [10.4.6 Predicting a sequence](#10.4.6)
    - [10.4.7 Generating a response](#10.4.7)
    - [10.4.8 Converse with your chatbot](#10.4.8)
* [10.5 Enhancements](#10.5)
    - [10.5.1 Reduce training complexity with bucketing](#10.5.1)
    - [10.5.2 Paying attention](#10.5.2)
* [10.6 In the real world](#10.6) -->

---
<a name='introduction'></a><a id='introduction'></a>
# Introduction
<a href="#top">[back to top]</a>

### Datasets

* fra-eng.zip: [script](#fra-eng.zip), [source](http://www.manythings.org/anki/fra-eng.zip)


### Explore

* Mapping one text sequence to another with a neural network.
* Understanding sequence-to-sequence tasks and how they're different from the others we've learned about.
* Using encoder-decoder model architectures for translation and chat.
* Training a model to pay attention to what is important in a sequence. 


### Key points

* Sequence-to-sequence networks can be built with a modular, reusable encoder-decoder architecture.
* The encoder model generates a thought vector, a dense, fixed-dimension vector representation of the information in a variable-length input sequence.
* A decoder can use thought vectors to predict (generate) output sequences, including the replies of a chatbot.
* Due to the thought vector representation, the input and output sequence lengths don't have to match.
* Thought vectors can only hold a limited amount of information. If you need a thought vector to encode more complex concepts, the attention mechanism can help selectively encode what is important in the thought vector.

---
<a name='10.0'></a><a id='10.0'></a>
# 10.0 Imports and Setup
<a href="#top">[back to top]</a>

In [1]:
import os
if not os.path.exists('setup'):
    os.mkdir('setup')

In [2]:
req_file = "setup/requirements_10.txt"

In [3]:
%%writefile {req_file}
isort
scikit-learn-intelex
scrapy
watermark

Overwriting setup/requirements_10.txt


In [4]:
import sys
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("Installing packages")
    !pip install --upgrade --quiet -r {req_file}
else:
    print("Running locally.")

Running locally.


In [5]:
#if IS_COLAB:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
%%writefile setup/chp10_imports.py
import locale
import os
import pprint
import random
import warnings

import numpy as np
import seaborn as sns
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import Dense
from tqdm.auto import tqdm
from watermark import watermark

Overwriting setup/chp10_imports.py


In [7]:
!isort setup/chp10_imports.py --sl
!cat setup/chp10_imports.py

Fixing /Users/gb/Desktop/examples/setup/chp10_imports.py
import locale
import os
import pprint
import random
import warnings

import numpy as np
import seaborn as sns
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import Dense
from tqdm.auto import tqdm
from watermark import watermark


In [8]:
import locale
import os
import pprint
import random
import warnings

import numpy as np
import seaborn as sns
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import Dense
from tqdm.auto import tqdm
from watermark import watermark

In [9]:
def HR():
    print("-"*40)
    
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
warnings.filterwarnings('ignore')
sns.set_style("darkgrid")
tqdm.pandas(desc="progress-bar")
pp = pprint.PrettyPrinter(indent=4)
random.seed(42)
np.random.seed(42)

print(watermark(iversions=True,globals_=globals(),python=True,machine=True))

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 7.34.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.3)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

numpy  : 1.23.5
sys    : 3.8.12 (default, Dec 13 2021, 20:17:08) 
[Clang 13.0.0 (clang-1300.0.29.3)]
seaborn: 0.12.1



---
<a name='10.1'></a><a id='10.1'></a>
# 10.1 Code
<a href="#top">[back to top]</a>

<a id='fra-eng.zip'></a><a name='fra-eng.zip'></a>
### Dataset: aclImdb_v1.tar.gz
<a href="#top">[back to top]</a>

In [10]:
# http://www.manythings.org/anki/
# http://www.manythings.org/anki/fra-eng.zip

In [11]:
data_fra_eng_dir = 'data/data_fra_eng'
if not os.path.exists(data_fra_eng_dir):
    os.makedirs(data_fra_eng_dir)
        
data_fra_eng_file = 'fra-eng.zip'
data_fra_eng_src = f"{data_fra_eng_dir}/{data_fra_eng_file}"
data_fra_eng_path = f"{data_fra_eng_dir}/fra.txt"
!wget -P {data_fra_eng_dir} -O {data_fra_eng_src} -nc "http://www.manythings.org/anki/fra-eng.zip"
!ls -l {data_fra_eng_dir}

File ‘data/data_fra_eng/fra-eng.zip’ already there; not retrieving.
total 75608
-rw-r--r--  1 gb  staff      1441 Feb 20 22:16 _about.txt
-rw-r--r--  1 gb  staff   7155035 Feb 21 08:53 fra-eng.zip
-rw-r--r--  1 gb  staff  31547877 Feb 20 22:16 fra.txt


In [12]:
print(data_fra_eng_src)
print(data_fra_eng_path)

if not os.path.exists(data_fra_eng_path):
    !unzip {data_fra_eng_src} -d {data_fra_eng_dir}

data/data_fra_eng/fra-eng.zip
data/data_fra_eng/fra.txt


In [13]:
batch_size = 64  # Batch size for training.
#epochs = 100  # Number of epochs to train for.
epochs = 1  # Number of epochs to train for.

latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10_000  # Number of samples to train on.

In [14]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_fra_eng_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [15]:
line[:50]

'Did I wake you?\tVous ai-je réveillés ?\tCC-BY 2.0 ('

In [16]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print(f"Number of samples: {len(input_texts):,}")
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
HR()
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
HR()

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print(type(encoder_input_data))
print(type(decoder_input_data))
print(type(decoder_target_data))

Number of samples: 10,000
Number of unique input tokens: 71
Number of unique output tokens: 93
----------------------------------------
Max sequence length for inputs: 15
Max sequence length for outputs: 59
----------------------------------------
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [17]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [18]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

2023-03-30 22:41:59.152719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# Run training
model.compile(
    optimizer='rmsprop', 
    loss='categorical_crossentropy'
)

hist = model.fit(
    [encoder_input_data, decoder_input_data], 
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=2
)

# Save model
# model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())



def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(
        input_seq,
        verbose=0
    )

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value,
            verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

125/125 - 37s - loss: 0.9723 - val_loss: 1.0528 - 37s/epoch - 298ms/step


In [20]:
print("Take one sequence (part of the training test for trying out decoding.")
print()

for seq_index in range(20):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    HR()
    
print("Done")

Take one sequence (part of the training test for trying out decoding.

Input sentence: Go.
Decoded sentence: ouee                                                        
----------------------------------------
Input sentence: Go.
Decoded sentence: ouee                                                        
----------------------------------------
Input sentence: Go.
Decoded sentence: ouee                                                        
----------------------------------------
Input sentence: Go.
Decoded sentence: ouee                                                        
----------------------------------------
Input sentence: Hi.
Decoded sentence: Toeee                                                       
----------------------------------------
Input sentence: Hi.
Decoded sentence: Toeee                                                       
----------------------------------------
Input sentence: Run!
Decoded sentence: oueee                                             